<a href="https://colab.research.google.com/github/kanishkjain1995/Learning-Tensorflow/blob/master/LearningTensorflow_(Ch_2_3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
print(tf.__version__)

1.14.0


## *Example 2.1* - *Hello World*

In [0]:
h = tf.constant("hello")
w = tf.constant("world!")

res = h + w

In [0]:
with tf.Session() as sess:
  print(sess.run(res))

b'helloworld!'


In [0]:
print(res)

Tensor("add:0", shape=(), dtype=string)


## *Example 2.2* - *MNIST Softmax Regression*

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
DATA_DIR = '/tmp/data'
NUM_ITERATIONS = 1000
MINIBATCH_SIZE = 100

In [0]:
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 784])
W = tf.Variable(tf.zeros(shape=[784, 10]))

In [0]:
y_true = tf.placeholder(dtype=tf.float32, shape=[None, 10])
y_pred = tf.matmul(x,W)

In [0]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [0]:
correct_pred = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for _ in range(NUM_ITERATIONS):
    x_train, y_train = data.train.next_batch(MINIBATCH_SIZE)
    sess.run(train_step, feed_dict={x:x_train, y_true:y_train})
  x_test, y_test = data.test.images, data.test.labels
  acc = sess.run(accuracy, feed_dict={x:x_test, y_true:y_test})
  
print("Accuracy is {:.2f}%".format(acc*100))

Accuracy is 91.98%


In [0]:
sess = tf.InteractiveSession()  
out = tf.random.truncated_normal([2,3])
print(out.eval())

[[-0.6196749   0.12019282  0.13404314]
 [ 0.66068035 -1.3794898   1.1542788 ]]


In [0]:
sess.close()

In [0]:
tf.constant(2.0)

<tf.Tensor 'Const_4:0' shape=() dtype=float32>

## *Placeholders*

In [0]:
with tf.Graph().as_default():
  x = tf.placeholder(tf.float32, shape=[5, 10])
  w = tf.placeholder(tf.float32, shape=[10,1])
  xw = tf.matmul(x,w)
  with tf.Session() as sess:
    output = sess.run(xw, feed_dict={x:np.random.rand(5,10), w:np.random.rand(10,1)})
  print("Output is {} and shape is {}".format(output, output.shape))

Output is [[1.4728774]
 [1.7282407]
 [1.7896023]
 [1.9331231]
 [1.3141056]] and shape is (5, 1)


## *Optimization*

### *Example* - *Linear Regression*

In [0]:
x_data = np.random.rand(1000,3)
w_real = [.3,.5,.2]
b_real = -.2

noise = np.random.rand(1, 1000)*0.01
y_data = np.matmul(w_real, x_data.T) + b_real + noise

In [0]:
NUM_ITER = 100

g = tf.Graph()

with g.as_default():
  x = tf.placeholder(tf.float32, shape=[None, 3])
  y_true = tf.placeholder(tf.float32, shape=None)
  
  with tf.name_scope('inference') as scope:
    w = tf.Variable([[0.,0.,0.]], name='weights')
    b = tf.Variable(0., name='bias')
    y_pred = tf.matmul(w, tf.transpose(x)) + b
  
  with tf.name_scope('loss') as scope:
    loss = tf.reduce_mean(tf.squared_difference(y_true, y_pred))
    
  with tf.name_scope('train') as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.5)
    train_step = optimizer.minimize(loss)
  
  
  init = tf.global_variables_initializer()
  with tf.Session() as sess:
    sess.run(init)
    for _ in range(NUM_ITER):
      sess.run(train_step, feed_dict={x:x_data, y_true:y_data})
    print("Update parameters={}".format(sess.run([w,b])))

Update parameters=[array([[0.2980278 , 0.49800313, 0.19836046]], dtype=float32), -0.19220619]


### *Example* - *Logisti Regression*

In [0]:
x_data = np.random.randn(20000,3)
w_real = [[0.3,0.5,0.1]]
b_real = -0.1

In [0]:
def sigmoid(z):
  return 1/(1+np.exp(-z))

In [0]:
y_data_pre_noise = sigmoid(np.matmul(w_real, x_data.T) + b_real)
y_data = np.random.binomial(1, y_data_pre_noise)

In [0]:
NUM_STEP = 50
g1 = tf.Graph()
with g1.as_default():
  x = tf.placeholder(tf.float32, shape=[None, 3])
  y_true = tf.placeholder(tf.float32, shape=None)
  
  with tf.name_scope('inference') as scope:
    w = tf.Variable([[0, 0, 0]], dtype=tf.float32, name='weights')
    b = tf.Variable(0, dtype=tf.float32, name='bias')
    y_pred = tf.sigmoid(tf.matmul(w, tf.transpose(x)) + b)
  
  with tf.name_scope('loss') as scope:
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    loss = tf.reduce_mean(loss)
    
  with tf.name_scope('train') as scope:
    learning_rate = 0.5
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(loss)
    
  init = tf.global_variables_initializer()
  with tf.Session() as sess:
    sess.run(init)
    for _ in range(NUM_STEP):
      sess.run(train, feed_dict={x:x_data, y_true:y_data})
      
    print("Parameters are {}".format(sess.run([w, b])))

Parameters are [array([[0.30706573, 0.5003172 , 0.11274564]], dtype=float32), -0.709256]
